# Markdown to Google Docs Converter

This notebook converts markdown meeting notes into a well-formatted Google Doc.

## Setup Instructions
1. Run each cell in order
2. Authenticate when prompted
3. The script will create a new Google Doc with formatted content

## Step 1: Install Dependencies

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib -q

## Step 2: Import Libraries and Authenticate

In [ ]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate with Google
auth.authenticate_user()

# Build the Docs API service
docs_service = build('docs', 'v1')
drive_service = build('drive', 'v3')

print("✓ Authentication successful!")

## Step 3: Define the Markdown Content

In [ ]:
MARKDOWN_CONTENT = """# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues
### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

print("✓ Markdown content loaded!")

## Step 4: Define the Converter Class

In [ ]:
class MarkdownToGoogleDocsConverter:
    """
    Converts markdown content to a formatted Google Doc.

    Supports:
    - Headings (H1, H2, H3)
    - Bullet points with nesting
    - Checkboxes
    - @mentions with bold styling
    - Footer styling
    """

    def __init__(self, docs_service, drive_service):
        """Initialize with Google API services."""
        self.docs_service = docs_service
        self.drive_service = drive_service
        self.document_id = None
        self.requests = []
        self.current_index = 1  # Google Docs index starts at 1

    def create_document(self, title):
        """
        Create a new Google Doc with the specified title.

        Args:
            title: The title for the new document

        Returns:
            The document ID of the created document
        """
        try:
            doc = self.docs_service.documents().create(
                body={'title': title}
            ).execute()
            self.document_id = doc.get('documentId')
            print(f"✓ Created document: {title}")
            print(f"  Document ID: {self.document_id}")
            return self.document_id
        except HttpError as e:
            raise Exception(f"Failed to create document: {e}")

    def parse_markdown(self, markdown_text):
        """
        Parse markdown text and generate Google Docs API requests.

        Args:
            markdown_text: The markdown content to parse
        """
        lines = markdown_text.strip().split('\n')
        i = 0

        while i < len(lines):
            line = lines[i]

            # Skip empty lines
            if not line.strip():
                i += 1
                continue

            # Handle horizontal rule (footer separator)
            if line.strip() == '---':
                self._add_horizontal_rule()
                i += 1
                continue

            # Handle headings
            if line.startswith('###'):
                self._add_heading(line[3:].strip(), 'HEADING_3')
            elif line.startswith('##'):
                self._add_heading(line[2:].strip(), 'HEADING_2')
            elif line.startswith('#'):
                self._add_heading(line[1:].strip(), 'HEADING_1')

            # Handle checkboxes (action items)
            elif re.match(r'^-\s*\[\s*\]', line):
                self._add_checkbox_item(line)

            # Handle bullet points (- or *)
            elif re.match(r'^(\s*)[-*]\s+', line):
                self._add_bullet_point(line)

            # Handle footer lines
            elif line.startswith('Meeting recorded by:') or line.startswith('Duration:'):
                self._add_footer_line(line)

            # Handle regular text
            else:
                self._add_text(line + '\n')

            i += 1

    def _add_heading(self, text, heading_style):
        """Add a heading with the specified style."""
        start_index = self.current_index
        text_with_newline = text + '\n'

        # Insert the text
        self.requests.append({
            'insertText': {
                'location': {'index': start_index},
                'text': text_with_newline
            }
        })

        # Apply heading style
        self.requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text_with_newline)
                },
                'paragraphStyle': {'namedStyleType': heading_style},
                'fields': 'namedStyleType'
            }
        })

        self.current_index += len(text_with_newline)

    def _add_bullet_point(self, line):
        """Add a bullet point with proper indentation."""
        # Calculate indentation level
        match = re.match(r'^(\s*)[-*]\s+(.+)$', line)
        if not match:
            return

        indent = len(match.group(1))
        text = match.group(2)
        nesting_level = indent // 2  # Each 2 spaces = 1 nesting level

        start_index = self.current_index

        # Process @mentions in text
        processed_text, mention_ranges = self._process_mentions(text, start_index)
        text_with_newline = processed_text + '\n'

        # Insert the text
        self.requests.append({
            'insertText': {
                'location': {'index': start_index},
                'text': text_with_newline
            }
        })

        # Apply bullet list formatting
        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text_with_newline)
                },
                'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
            }
        })

        # Apply indentation for nested items
        if nesting_level > 0:
            self.requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': start_index,
                        'endIndex': start_index + len(text_with_newline)
                    },
                    'paragraphStyle': {
                        'indentStart': {'magnitude': 36 * nesting_level, 'unit': 'PT'},
                        'indentFirstLine': {'magnitude': 36 * nesting_level, 'unit': 'PT'}
                    },
                    'fields': 'indentStart,indentFirstLine'
                }
            })

        # Apply bold styling to @mentions
        for mention_start, mention_end in mention_ranges:
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': mention_start,
                        'endIndex': mention_end
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {'red': 0.0, 'green': 0.4, 'blue': 0.8}
                            }
                        }
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

        self.current_index += len(text_with_newline)

    def _add_checkbox_item(self, line):
        """Add a checkbox item (action item)."""
        # Extract content after checkbox
        match = re.match(r'^-\s*\[\s*\]\s*(.+)$', line)
        if not match:
            return

        text = match.group(1)
        start_index = self.current_index

        # Process @mentions
        processed_text, mention_ranges = self._process_mentions(text, start_index)
        text_with_newline = processed_text + '\n'

        # Insert the text
        self.requests.append({
            'insertText': {
                'location': {'index': start_index},
                'text': text_with_newline
            }
        })

        # Apply checkbox formatting
        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text_with_newline)
                },
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })

        # Apply bold and color styling to @mentions
        for mention_start, mention_end in mention_ranges:
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': mention_start,
                        'endIndex': mention_end
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {'red': 0.0, 'green': 0.4, 'blue': 0.8}
                            }
                        }
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

        self.current_index += len(text_with_newline)

    def _process_mentions(self, text, base_index):
        """
        Find @mentions in text and return their positions.

        Args:
            text: The text to process
            base_index: The starting index in the document

        Returns:
            Tuple of (text, list of (start, end) ranges for mentions)
        """
        mention_ranges = []
        # Find all @mentions (e.g., @sarah, @mike)
        for match in re.finditer(r'@\w+', text):
            start = base_index + match.start()
            end = base_index + match.end()
            mention_ranges.append((start, end))
        return text, mention_ranges

    def _add_horizontal_rule(self):
        """Add a horizontal rule (visual separator)."""
        separator = '─' * 50 + '\n'
        start_index = self.current_index

        self.requests.append({
            'insertText': {
                'location': {'index': start_index},
                'text': separator
            }
        })

        # Style the separator
        self.requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(separator)
                },
                'textStyle': {
                    'foregroundColor': {
                        'color': {
                            'rgbColor': {'red': 0.6, 'green': 0.6, 'blue': 0.6}
                        }
                    }
                },
                'fields': 'foregroundColor'
            }
        })

        self.current_index += len(separator)

    def _add_footer_line(self, line):
        """Add footer text with distinct styling (italic, gray)."""
        start_index = self.current_index
        text_with_newline = line + '\n'

        self.requests.append({
            'insertText': {
                'location': {'index': start_index},
                'text': text_with_newline
            }
        })

        # Apply footer styling (italic and gray)
        self.requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text_with_newline)
                },
                'textStyle': {
                    'italic': True,
                    'foregroundColor': {
                        'color': {
                            'rgbColor': {'red': 0.5, 'green': 0.5, 'blue': 0.5}
                        }
                    }
                },
                'fields': 'italic,foregroundColor'
            }
        })

        self.current_index += len(text_with_newline)

    def _add_text(self, text):
        """Add plain text."""
        start_index = self.current_index

        self.requests.append({
            'insertText': {
                'location': {'index': start_index},
                'text': text
            }
        })

        self.current_index += len(text)

    def apply_formatting(self):
        """
        Execute all batched requests to format the document.

        Returns:
            The API response
        """
        if not self.requests:
            print("No formatting requests to apply.")
            return None

        try:
            result = self.docs_service.documents().batchUpdate(
                documentId=self.document_id,
                body={'requests': self.requests}
            ).execute()
            print(f"✓ Applied {len(self.requests)} formatting requests")
            return result
        except HttpError as e:
            raise Exception(f"Failed to apply formatting: {e}")

    def get_document_url(self):
        """Get the URL to view the created document."""
        return f"https://docs.google.com/document/d/{self.document_id}/edit"

    def convert(self, markdown_text, title="Converted Document"):
        """
        Main method to convert markdown to Google Doc.

        Args:
            markdown_text: The markdown content to convert
            title: The title for the new document

        Returns:
            The URL of the created document
        """
        print("\n" + "="*50)
        print("Starting Markdown to Google Docs Conversion")
        print("="*50 + "\n")

        # Step 1: Create the document
        self.create_document(title)

        # Step 2: Parse markdown and build requests
        print("✓ Parsing markdown content...")
        self.parse_markdown(markdown_text)

        # Step 3: Apply all formatting
        self.apply_formatting()

        # Step 4: Get and return the document URL
        url = self.get_document_url()
        print("\n" + "="*50)
        print("Conversion Complete!")
        print("="*50)
        print(f"\n📄 View your document at:\n{url}\n")

        return url

print("✓ Converter class defined!")

## Step 5: Run the Conversion

In [ ]:
# Create converter instance and run the conversion
converter = MarkdownToGoogleDocsConverter(docs_service, drive_service)

try:
    # Convert the markdown to a Google Doc
    document_url = converter.convert(
        markdown_text=MARKDOWN_CONTENT,
        title="Product Team Sync - May 15, 2023"
    )

    # Display clickable link
    from IPython.display import display, HTML
    display(HTML(f'<a href="{document_url}" target="_blank">Click here to open your Google Doc</a>'))

except Exception as e:
    print(f"❌ Error during conversion: {e}")
    print("\nTroubleshooting tips:")
    print("1. Make sure you've authenticated successfully")
    print("2. Check that you have Google Docs API enabled")
    print("3. Verify your Google account has permission to create documents")